## RDDs and DataFrames

* Creating RDDs and DataFrames using SparkContext
* Interoperability between RDDs and DataFrames
* Multiple rows and multiple column specifications for DataFrames
* Creating DataFrames using SQLContext
* Selecting, editing and renaming columns in dataframes
* Interoperability between Pandas and Spark dataframes

In [1]:
# check if Spark works after install on local machine
from pyspark import SparkConf, SparkContext
# sc = SparkContext("local","test")
# sc.parallelize([1,2,3,4])
# sc.stop()


ModuleNotFoundError: No module named 'pyspark'

In [3]:
%%cmd
java -version

Microsoft Windows [Version 10.0.17763.737]
(c) 2018 Microsoft Corporation. All rights reserved.

(base) C:\Users\NL27514\Documents\Atos\Projects\ITPH\Wk 2 D 2\Pyspark>java -version

(base) C:\Users\NL27514\Documents\Atos\Projects\ITPH\Wk 2 D 2\Pyspark>

'java' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
# TEST connection to cluster running on VM

# go to the spark web UI in the browser and search for spark.driver.host and spark.driver.port under the Environment tab
conf = SparkConf().setAppName('hello2').setMaster('spark://192.168.56.101:42691')  # fill found info here
sc = SparkContext(conf=conf)

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("spark://192.168.56.101:7070") \
    .appName("Day1") \
    .getOrCreate()



In [ ]:
sc = spark.sparkContext
sc.uiWebUrl

Here we use SparkContext. Normally, SparkSession is used.
Sparksession encapsulated Sparkcontext.
- simplified entry point
- no confusion on which context to uses (sql, Hives etc)
![](sparksession.jpg)


In [ ]:
from pyspark.sql.types import Row
from datetime import datetime

#### Creating RDDs using sc.parallelize()
In this example different types of data are used, being string and integer


In [ ]:
simple_data = sc.parallelize([1, "Alice", 50])
simple_data

Parallelized collections are created by calling SparkContext’s parallelize method on an existing iterable or collection in your driver program. The elements of the collection are copied to form a distributed dataset that can be operated on in parallel. For example, here is how to create a parallelized collection holding the numbers 1 to 5:

<p>data = [1, 2, 3, 4, 5] </p>

distData = sc.parallelize(data)

Once created, the distributed dataset (distData) can be operated on in parallel. For example, we can call distData.reduce(lambda a, b: a + b) to add up the elements of the list. We describe operations on distributed datasets later on.

One important parameter for parallel collections is the number of partitions to cut the dataset into. Spark will run one task for each partition of the cluster. Typically you want 2-4 partitions for each CPU in your cluster. Normally, Spark tries to set the number of partitions automatically based on your cluster. However, you can also set it manually by passing it as a second parameter to parallelize (e.g. sc.parallelize(data, 10)). Note: some places in the code use the term slices (a synonym for partitions) to maintain backward compatibility.

In [ ]:
simple_data.count()

![](count.png)

In [ ]:
simple_data.first()

Remember: Count() en First() are ACTIONS on the RDD dataset.


In [ ]:
simple_data.take(2)

(takes the first 2 elements of the dataset)

![](collect.png)

In [ ]:
simple_data.collect()

The collect Action, shows all the elements in de dataset

#### ERROR

* This RDD does not have "columns", it cannot be represented as a tabular data frame
* DataFrames are structured datasets
* So, instead of working with RDD, we want to work with Dataframes.

Error: This RDD has NO schema, contains elements of different types - it cannot be converted to a Dataframe.

In [ ]:
df = simple_data.toDF()

#### RDDs with records using sc.parallelize()

In [ ]:
records = sc.parallelize([[1, "Alice", 50], [2, "Bob", 80]])
# Notice: this has 2 record, one for Alice, and one for Bob
records  
# The RDD is now created

In [ ]:
records.collect()

In [ ]:
df = records.toDF()

In [ ]:
df

In [ ]:
df.show()
# Column names have been automatically generated and assigned

In [ ]:

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()
records = spark.sparkContext.parallelize([[1, "Alice", 50], [2, "Bob", 80]])
help(sc.parallelize)

In [ ]:
# spark.sql("SET -v").show(n=200, truncate=False)

The method show will present the first 20 (by default) rows.

In [ ]:
df.show()

#### Creating dataframes using sc.parallelize() and Row() functions
* Row functions allow specifying column names for dataframes

In [ ]:
data = sc.parallelize([Row(id=1,
                           name="Alice",
                           score=50)])
# Row is imported in the first lines of this notebook
data

In [ ]:
data.count()

In [ ]:
data.collect()

In [ ]:
df = data.toDF()
df.show()

So: Structured data with a SCHEMA can be converted to a Dataframe

#### Working with multiple rows

In [ ]:
data = sc.parallelize([Row(
                           id=1,
                           name="Alice",
                           score=50
                        ),
                        Row(
                            id=2,
                            name="Bob",
                            score=80
                        ),
                        Row(
                            id=3,
                            name="Charlee",
                            score=75
                        )])

In [ ]:
df = data.toDF()
df.show()

#### Multiple columns with complex data types

In [ ]:
complex_data = sc.parallelize([Row(
                                col_float=1.44,
                                col_integer=10,
                                col_string="John")
                           ])

In [ ]:
complex_data_df = complex_data.toDF()
complex_data_df.show()

In [ ]:
complex_data = sc.parallelize([Row(
                                col_float=1.44, 
                                col_integer=10, 
                                col_string="John", 
                                col_boolean=True, 
                                col_list=[1, 2, 3])
                           ])
# The last Column is a List....

In [ ]:
complex_data_df = complex_data.toDF()
complex_data_df.show()

In [ ]:
complex_data = sc.parallelize([Row(
                                col_list = [1, 2, 3], 
                                col_dict = {"k1": 0, "k2": 1, "k3": 2}, 
                                col_row = Row(columnA = 10, columnB = 20, columnC = 30), 
                                col_time = datetime(2014, 8, 1, 14, 1, 5)
                            )])

In [ ]:
complex_data_df = complex_data.toDF()
complex_data_df.show()

#### Multiple rows with complex data types

In [ ]:
complex_data = sc.parallelize([Row(
                                col_list = [1, 2, 3],
                                col_dict = {"k1": 0},
                                col_row = Row(a=10, b=20, c=30),
                                col_time = datetime(2014, 8, 1, 14, 1, 5)
                            ),              
                            Row(
                                col_list = [1, 2, 3, 4, 5], 
                                col_dict = {"k1": 0,"k2": 1 }, 
                                col_row = Row(a=40, b=50, c=60),
                                col_time = datetime(2014, 8, 2, 14, 1, 6)
                            ),
                            Row(
                                col_list = [1, 2, 3, 4, 5, 6, 7], 
                                col_dict = {"k1": 0, "k2": 1, "k3": 2 }, 
                                col_row = Row(a=70, b=80, c=90),
                                col_time = datetime(2014, 8, 3, 14, 1, 7)
                            )]) 

In [ ]:
complex_data_df = complex_data.toDF()
complex_data_df.show()

#### Creating DataFrames using SQLContext

* SQLContext can create dataframes directly from raw data

https://spark.apache.org/docs/1.6.1/sql-programming-guide.html

<p>Example</p>:
<p>Create directly from JSON</p>
<p>sqlContext = SQLContext(sc)</p>
<p>df = sqlContext.read.json("examples/src/main/resources/people.json")</p>

In [ ]:
from pyspark.sql import SQLContext

In [ ]:
# sqlContext  = SQLContext(spark.sparkContext)
sqlContext = SQLContext(sc)
help(sqlContext)

In [ ]:
sqlContext

In [ ]:
df = sqlContext.range(5)
df

In [ ]:
df.show()

In [ ]:
df.count()

#### Rows specified in tuples

In [ ]:
data = [('Alice', 50),
        ('Bob', 80),
        ('Charlee', 75)]

In [ ]:
sqlContext.createDataFrame(data).show()

Specify explicitely column names!

In [ ]:
sqlContext.createDataFrame(data, ['Name', 'Score']).show()

In [ ]:
complex_data = [
                 (1.0,
                  10,
                  "Alice", 
                  True, 
                  [1, 2, 3], 
                  {"k1": 0},
                  Row(a=1, b=2, c=3), 
                  datetime(2014, 8, 1, 14, 1, 5)),

                 (2.0,
                  20,
                  "Bob", 
                  True, 
                  [1, 2, 3, 4, 5], 
                  {"k1": 0,"k2": 1 }, 
                  Row(a=1, b=2, c=3), 
                  datetime(2014, 8, 1, 14, 1, 5)),

                  (3.0,
                   30,
                   "Charlee", 
                   False, 
                   [1, 2, 3, 4, 5, 6], 
                   {"k1": 0, "k2": 1, "k3": 2 }, 
                   Row(a=1, b=2, c=3), 
                   datetime(2014, 8, 1, 14, 1, 5))
                ] 

In [ ]:
sqlContext.createDataFrame(complex_data).show()

In [ ]:
complex_data_df = sqlContext.createDataFrame(complex_data, [
        'col_integer',
        'col_float',
        'col_string',
        'col_boolean',
        'col_list',
        'col_dictionary',
        'col_row',
        'col_date_time']
    )
complex_data_df.show()

#### Creating dataframes using SQL Context and the Row function
* Row functions can be used without specifying column names

In [ ]:
data = sc.parallelize([
    Row(1, "Alice", 50),
    Row(2, "Bob", 80),
    Row(3, "Charlee", 75)
])


You can always setup column names, as explained below:

In [ ]:
column_names = Row('id', 'name', 'score')  
students = data.map(lambda r: column_names(*r))
# Lambda's : functional programming within on OO world....VERY HANDY, but VERY confusing.....

# The map() operations performs a transformation on every element in the RDD.qw


In [ ]:
students

In [ ]:
students.collect()

In [ ]:
students_df = sqlContext.createDataFrame(students)
students_df

In [ ]:
students_df.show()

#### Extracting specific rows from dataframes

In [ ]:
complex_data_df.first()

In [ ]:
complex_data_df.take(2)

#### Extracting specific cells from dataframes

In [ ]:
cell_string = complex_data_df.collect()[0][2]
cell_string

In [ ]:
cell_list = complex_data_df.collect()[0][4]
cell_list

In [ ]:
cell_list.append(100)
cell_list

In [ ]:
# The orignional dataframe has not been modified
# When you access a cell, that cell will be unchanged
complex_data_df.show()

#### Selecting specific columns

Every Dataframe contains the RDD equivalent of the record that it stores, in the RDD variable

In [ ]:
complex_data_df.rdd\
    .map(lambda x: (x.col_string, x.col_dictionary))\
    .collect()

In [ ]:
complex_data_df.select(
    'col_string',
    'col_list',
    'col_date_time'
).show()

#### Editing columns

In [ ]:
# A map() operation which appends "Boo" to every string in the column
complex_data_df.rdd\
           .map(lambda x: (x.col_string + " Boo"))\
           .collect()
# Dataframes do not support the add function. To do this, you must create a new dataframe

#### Adding a column

In [ ]:
complex_data_df.select(
                   'col_integer',
                   'col_float'
            )\
           .withColumn(
                   "col_sum",
                    complex_data_df.col_integer + complex_data_df.col_float
           )\
           .show()

Another example, producing the opposite boolean column

In [ ]:
complex_data_df.select('col_boolean')\
               .withColumn(
                   "col_opposite",
                   complex_data_df.col_boolean == False )\
               .show()

#### Editing a column name / Renaming

In [ ]:
complex_data_df.withColumnRenamed("col_dictionary","col_map").show()
# A new dataframe is actually made

In [ ]:
complex_data_df.select(complex_data_df.col_string.alias("Name")).show()

#### Interoperablity between Pandas dataframe and Spark dataframe
Remember: RDD and Dataframes are distributed accros nodes. Pandas will be on ONE machine (in memory).

In [ ]:
import pandas

In [ ]:
df_pandas = complex_data_df.toPandas()
df_pandas

From Pandas to Dataframe....

In [ ]:
df_spark = sqlContext.createDataFrame(df_pandas).show()  
df_spark